# Will XGBoost trained in segmented words surrounding the mention of a risk factor work to classify notes into "yes, risk factor" and "no, risk factor"?

In [ ]:
# Generic imports
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import pandas as pd
import numpy as np
import json
import pickle
from tqdm import tqdm   # For keeping track of loops
from pathlib import Path
from joblib import Parallel, delayed

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Custom imports
import sys
sys.path.append("../")
from src.diagnosis_tools import *
import src.plots as plots
from custom_functions import *

In [ ]:
# set plotting params
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.reload_library()
rcparams = plots.stdrcparams1()
mpl.rcParams.update(rcparams)

In [ ]:
# Models/algorithms/classifiers
from xgboost import XGBClassifier

# Evaluation of models
from sklearn.model_selection import KFold
from sklearn.metrics import *

# Text vectorizer
from sklearn.feature_extraction.text import CountVectorizer

import shap

In [ ]:
# Data locations
basedir = Path("../..")
analysis_location = basedir / 'Analysis_Data'
cohort = 'hospital_a_2013'
path = analysis_location / cohort
training_path = analysis_location / "train_ML"
mimic3_path = analysis_location / "MIMIC_III" / "labeled_subset"

# Figures
figure_path = basedir / "Figures"

### Reading in the files

In [ ]:
notes_annot = pd.read_csv(path / "attending_notes_annotated.csv")
notes_annot['notes_timestamp'] = pd.to_timedelta(notes_annot['notes_timestamp'])

# Reading in MIMIC III
mimic_iii = pd.read_csv(mimic3_path / "attending_notes.csv")
mimic_iii['notes_timestamp'] = pd.to_datetime(mimic_iii['notes_timestamp'])

### Segmenting each relevant target

In [ ]:
# Hospital A (2013)
valid_pna = notes_annot['seg_pneumonia'] != "Invalid"
valid_chf = notes_annot['seg_chf'] != "Invalid"
valid_aspiration = notes_annot['seg_aspiration'] != "Invalid"
valid_sepsis = notes_annot['seg_sepsis'] != "Invalid"

# MIMIC III
valid_pna_mimic = mimic_iii['seg_pneumonia'] != "Invalid"

In [ ]:
# Subsetting tables to only have entries that matched, and relevant columns. Hospital A (2013)
pneumonia_notes = notes_annot.loc[valid_pna,
                                  ['encounter_id', 'notes_timestamp', "notes_text",
                                   'pneumonia', 'pneumonia_sw', 'seg_pneumonia']]
pneumonia_notes['seg_pneumonia'] = pneumonia_notes['seg_pneumonia'].str.replace(r"'", r"", regex=True)
pneumonia_notes['seg_pneumonia'] = pneumonia_notes['seg_pneumonia'].str.replace(r"\[", r"", regex=True)
pneumonia_notes['seg_pneumonia'] = pneumonia_notes['seg_pneumonia'].str.replace(r"\]", r"", regex=True)
pneumonia_notes['seg_pneumonia'] = pneumonia_notes['seg_pneumonia'].str.replace(r",", r"", regex=True)

chf_notes = notes_annot.loc[valid_chf,
                            ['encounter_id', 'notes_timestamp', "notes_text",
                             'chf', 'chf_sw', 'seg_chf']]
chf_notes['seg_chf'] = chf_notes['seg_chf'].str.replace(r"'", r"", regex=True)
chf_notes['seg_chf'] = chf_notes['seg_chf'].str.replace(r"\[", r"", regex=True)
chf_notes['seg_chf'] = chf_notes['seg_chf'].str.replace(r"\]", r"", regex=True)
chf_notes['seg_chf'] = chf_notes['seg_chf'].str.replace(r",", r"", regex=True)

aspiration_notes = notes_annot.loc[valid_aspiration,
                                   ['encounter_id', 'notes_timestamp', "notes_text",
                                    'aspiration', 'aspiration_sw', 'seg_aspiration']]
aspiration_notes['seg_aspiration'] = aspiration_notes['seg_aspiration'].str.replace(r"'", r"", regex=True)
aspiration_notes['seg_aspiration'] = aspiration_notes['seg_aspiration'].str.replace(r"\[", r"", regex=True)
aspiration_notes['seg_aspiration'] = aspiration_notes['seg_aspiration'].str.replace(r"\]", r"", regex=True)
aspiration_notes['seg_aspiration'] = aspiration_notes['seg_aspiration'].str.replace(r",", r"", regex=True)

sepsis_notes = notes_annot.loc[valid_sepsis,
                               ['encounter_id', 'notes_timestamp', "notes_text",
                                'sepsis', 'sepsis_sw', 'seg_sepsis']]
sepsis_notes['seg_sepsis'] = sepsis_notes['seg_sepsis'].str.replace(r"'", r"", regex=True)
sepsis_notes['seg_sepsis'] = sepsis_notes['seg_sepsis'].str.replace(r"\[", r"", regex=True)
sepsis_notes['seg_sepsis'] = sepsis_notes['seg_sepsis'].str.replace(r"\]", r"", regex=True)
sepsis_notes['seg_sepsis'] = sepsis_notes['seg_sepsis'].str.replace(r",", r"", regex=True)

# MIMIC III
pneumonia_notes_mimic = mimic_iii.loc[valid_pna_mimic,
                                      ['encounter_id', 'notes_timestamp', "notes_text",
                                       'curt_pneumonia_(1=yes)', 'seg_pneumonia']]
pneumonia_notes_mimic['seg_pneumonia'] = pneumonia_notes_mimic['seg_pneumonia'].str.replace(r"'", r"", regex=True)
pneumonia_notes_mimic['seg_pneumonia'] = pneumonia_notes_mimic['seg_pneumonia'].str.replace(r"\[", r"", regex=True)
pneumonia_notes_mimic['seg_pneumonia'] = pneumonia_notes_mimic['seg_pneumonia'].str.replace(r"\]", r"", regex=True)
pneumonia_notes_mimic['seg_pneumonia'] = pneumonia_notes_mimic['seg_pneumonia'].str.replace(r",", r"", regex=True)

#### This is probably to get counts

In [ ]:
notes_for_ML = notes_annot.loc[valid_pna | valid_chf | valid_aspiration | valid_sepsis]

In [ ]:
notes_for_ML.encounter_id.nunique()

In [ ]:
notes_for_ML.shape

In [ ]:
len(pneumonia_notes), len(chf_notes), len(aspiration_notes), len(sepsis_notes)

In [ ]:
notes_annot.pneumonia_sw.value_counts(dropna=False)

In [ ]:
notes_annot.chf_sw.value_counts(dropna=False)

In [ ]:
notes_annot.aspiration_sw.value_counts(dropna=False)

In [ ]:
notes_annot.sepsis_sw.value_counts(dropna=False)

In [ ]:
pneumonia_notes['pneumonia_sw'].value_counts(dropna=False, normalize=True)

In [ ]:
chf_notes['chf_sw'].value_counts(dropna=False, normalize=True)

In [ ]:
aspiration_notes['aspiration_sw'].value_counts(dropna=False, normalize=True)

In [ ]:
sepsis_notes['sepsis_sw'].value_counts(dropna=False, normalize=True)

In [ ]:
# Imputing null SW adjudications as zero
pneumonia_notes['pneumonia_sw'] = pneumonia_notes['pneumonia_sw'].fillna(0)
chf_notes['chf_sw'] = chf_notes['chf_sw'].fillna(0)
aspiration_notes['aspiration_sw'] = aspiration_notes['aspiration_sw'].fillna(0)
sepsis_notes['sepsis_sw'] = sepsis_notes['sepsis_sw'].fillna(0)

In [ ]:
# pneumonia_notes.to_csv(training_path/"pneumonia_attending_notes.csv", index=False)

In [ ]:
# sepsis_notes.to_csv(training_path/"sepsis_attending_notes.csv", index=False)

In [ ]:
cols = {'a': ['seg_pneumonia', 'pneumonia_sw'],
        'b': ['seg_chf', 'chf_sw'],
        'c': ['seg_aspiration', 'aspiration_sw'],
        'd': ['seg_sepsis', 'sepsis_sw']}

#### Reading in hyperparameters

In [ ]:
with open(basedir / "Development_notebooks" /  "hyperparameters" / "pna_XG_hyperparams.json", "r") as pna_file:
    pna_hyperparams = json.load(pna_file)

pna_hyperparams['base_score'] = float(pna_hyperparams['base_score'])     
pna_hyperparams['n_estimators'] = int(pna_hyperparams['n_estimators'])
pna_hyperparams['max_depth'] = int(pna_hyperparams['max_depth'])
pna_hyperparams['learning_rate'] = float(pna_hyperparams['learning_rate'])
pna_hyperparams['gamma'] = float(pna_hyperparams['gamma'])
pna_hyperparams['min_child_weight'] = float(pna_hyperparams['min_child_weight'])
pna_hyperparams['max_delta_step'] = float(pna_hyperparams['max_delta_step'])
pna_hyperparams['subsample'] = float(pna_hyperparams['subsample'])

## Generating cross-validated AUCs and feature importances

In [ ]:
n_boot = 100

auc_pna = []
tprs_pna = []
pna_fops = []
pna_dws = []
importances_pna = []

auc_chf = []
tprs_chf = []
chf_fops = []
chf_dws = []
importances_chf = []

auc_aspiration = []
tprs_aspiration = []
aspiration_fops = []
aspiration_dws = []
importances_aspiration = []

auc_sepsis = []
tprs_sepsis = []
sepsis_fops = []
sepsis_dws = []
importances_sepsis = []

mean_fpr = np.arange(0, 1.01, 0.01)
mean_mpv = np.arange(0, 1.1, 0.1)

In [ ]:
for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_pneumonia_notes = pneumonia_notes.sample(n=len(pneumonia_notes), replace=True, axis=0)
    boot_chf_notes = chf_notes.sample(n=len(chf_notes), replace=True, axis=0)
    boot_aspiration_notes = aspiration_notes.sample(n=len(aspiration_notes), replace=True, axis=0)
    boot_sepsis_notes = sepsis_notes.sample(n=len(sepsis_notes), replace=True, axis=0)
    
    pneumonia_enctrs = boot_pneumonia_notes['encounter_id'].unique()
    chf_enctrs = boot_chf_notes['encounter_id'].unique()
    aspiration_enctrs = boot_aspiration_notes['encounter_id'].unique()
    sepsis_enctrs = boot_sepsis_notes['encounter_id'].unique()
    
    cv = KFold()
    
    # Training each model    
    # Pneumonia
    print("Pneumonia")
    output1, output2, output3, output4, output5 = Parallel(n_jobs=-1)(delayed(nested_cv)(
        boot_pneumonia_notes,
        cols,
        'a',
        pneumonia_enctrs,
        train_index,
        test_index,
        mean_fpr,
        mean_mpv,
        model="XGBoost"
        ) for train_index, test_index in cv.split(pneumonia_enctrs))
    
    auc_pna.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    tprs_pna.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    pna_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    pna_dws.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    importances_pna.extend(pd.DataFrame(
        output1[5]+output2[5]+output3[5]+output4[5]+output5[5]).groupby('feature').mean().reset_index().to_dict(orient='records'))
    
    
    # Congestive Heart Failure
    print("CHF")
    output1, output2, output3, output4, output5 = Parallel(n_jobs=-1)(delayed(nested_cv)(
        boot_chf_notes,
        cols,
        'b',
        chf_enctrs,
        train_index,
        test_index,
        mean_fpr,
        mean_mpv,
        model="XGBoost"
        ) for train_index, test_index in cv.split(chf_enctrs))
    
    auc_chf.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    tprs_chf.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    chf_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    chf_dws.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    importances_chf.extend(pd.DataFrame(
        output1[5]+output2[5]+output3[5]+output4[5]+output5[5]).groupby('feature').mean().reset_index().to_dict(orient='records'))
    
    
    # Aspiration
    print("Aspiration")
    output1, output2, output3, output4, output5 = Parallel(n_jobs=-1)(delayed(nested_cv)(
        boot_aspiration_notes,
        cols,
        'c',
        aspiration_enctrs,
        train_index,
        test_index,
        mean_fpr,
        mean_mpv,
        model="XGBoost"
        ) for train_index, test_index in cv.split(aspiration_enctrs))
    
    auc_aspiration.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    tprs_aspiration.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    aspiration_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    aspiration_dws.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    importances_aspiration.extend(pd.DataFrame(
        output1[5]+output2[5]+output3[5]+output4[5]+output5[5]).groupby('feature').mean().reset_index().to_dict(orient='records'))
    
    
    # Sepsis
    print("Sepsis")
    output1, output2, output3, output4, output5 = Parallel(n_jobs=-1)(delayed(nested_cv)(
        boot_sepsis_notes,
        cols,
        'd',
        sepsis_enctrs,
        train_index,
        test_index,
        mean_fpr,
        mean_mpv,
        model="XGBoost"
        ) for train_index, test_index in cv.split(sepsis_enctrs))

    auc_sepsis.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    tprs_sepsis.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    sepsis_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    sepsis_dws.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    importances_sepsis.extend(pd.DataFrame(
        output1[5]+output2[5]+output3[5]+output4[5]+output5[5]).groupby('feature').mean().reset_index().to_dict(orient='records'))

In [ ]:
# pna_data = {
#     "pna_auc": auc_pna,
#     "pna_tprs": tprs_pna,
#     "pna_fops": pna_fops,
#     "pna_dws": pna_dws,
#     "pna_importances": importances_pna,
# }

# chf_data = {
#     "chf_auc": auc_chf,
#     "chf_tprs": tprs_chf,
#     "chf_fops": chf_fops,
#     "chf_dws": chf_dws,
#     "chf_importances": importances_chf,
# }

# aspiration_data = {
#     "aspiration_auc": auc_aspiration,
#     "aspiration_tprs": tprs_aspiration,
#     "aspiration_fops": aspiration_fops,
#     "aspiration_dws": aspiration_dws,
#     "aspiration_importances": importances_aspiration,
# }

# sepsis_data = {
#     "sepsis_auc": auc_sepsis,
#     "sepsis_tprs": tprs_sepsis,
#     "sepsis_fops": sepsis_fops,
#     "sepsis_dws": sepsis_dws,
#     "sepsis_importances": importances_sepsis,
# }


# with open("pna_data.pkl", "wb") as file:
#     pickle.dump(pna_data, file)
    
# with open("chf_data.pkl", "wb") as file:
#     pickle.dump(chf_data, file)
    
# with open("aspiration_data.pkl", "wb") as file:
#     pickle.dump(aspiration_data, file)
    
# with open("sepsis_data.pkl", "wb") as file:
#     pickle.dump(sepsis_data, file)

In [ ]:
# Now we can load the data from the pickle files
with open("pna_data.pkl", "rb") as file:
    pna_data = pickle.load(file)
    
with open("chf_data.pkl", "rb") as file:
    chf_data = pickle.load(file)
    
with open("aspiration_data.pkl", "rb") as file:
    aspiration_data = pickle.load(file)
    
with open("sepsis_data.pkl", "rb") as file:
    sepsis_data = pickle.load(file)
    
auc_pna = pna_data['pna_auc']
tprs_pna = pna_data['pna_tprs']
pna_fops = pna_data['pna_fops']
pna_dws = pna_data['pna_dws']
importances_pna = pna_data['pna_importances']

auc_chf = chf_data['chf_auc']
tprs_chf = chf_data['chf_tprs']
chf_fops = chf_data['chf_fops']
chf_dws = chf_data['chf_dws']
importances_chf = chf_data['chf_importances']

auc_aspiration = aspiration_data['aspiration_auc']
tprs_aspiration = aspiration_data['aspiration_tprs']
aspiration_fops = aspiration_data['aspiration_fops']
aspiration_dws = aspiration_data['aspiration_dws']
importances_aspiration = aspiration_data['aspiration_importances']

auc_sepsis = sepsis_data['sepsis_auc']
tprs_sepsis = sepsis_data['sepsis_tprs']
sepsis_fops = sepsis_data['sepsis_fops']
sepsis_dws = sepsis_data['sepsis_dws']
importances_sepsis = sepsis_data['sepsis_importances']

#### Modifying data collected in the loop, for plotting

In [ ]:
# Pneumonia
mean_pna_tpr = np.mean(tprs_pna, axis=0)
mean_pna_tpr[-1] = 1.0
pna_tpr_CI95 = [
        np.percentile(tprs_pna, 2.5, axis=0),
        np.percentile(tprs_pna, 97.5, axis=0)
        ]

mean_pna_auc = np.mean(auc_pna)
pna_auc_CI95 = [
        np.percentile(auc_pna, 2.5),
        np.percentile(auc_pna, 97.5)
        ]

pna_importances_df = pd.DataFrame(importances_pna)

mean_pna_fop = np.mean(pna_fops, axis=0)
pna_fop_CI95 = [
        np.percentile(pna_fops, 2.5, axis=0),
        np.percentile(pna_fops, 97.5, axis=0)
        ]
mean_pna_dws = np.mean(pna_dws, axis=0)
pna_dws_CI95 = [
        np.percentile(pna_dws, 2.5, axis=0),
        np.percentile(pna_dws, 97.5, axis=0)
        ]


# Congestive Heart Failure
mean_chf_tpr = np.mean(tprs_chf, axis=0)
mean_chf_tpr[-1] = 1.0
chf_tpr_CI95 = [
        np.percentile(tprs_chf, 2.5, axis=0),
        np.percentile(tprs_chf, 97.5, axis=0)
        ]

mean_chf_auc = np.mean(auc_chf)
chf_auc_CI95 = [
        np.percentile(auc_chf, 2.5),
        np.percentile(auc_chf, 97.5)
        ]

chf_importances_df = pd.DataFrame(importances_chf)

mean_chf_fop = np.mean(chf_fops, axis=0)
chf_fop_CI95 = [
        np.percentile(chf_fops, 2.5, axis=0),
        np.percentile(chf_fops, 97.5, axis=0)
        ]
mean_chf_dws = np.mean(chf_dws, axis=0)
chf_dws_CI95 = [
        np.percentile(chf_dws, 2.5, axis=0),
        np.percentile(chf_dws, 97.5, axis=0)
        ]


# Aspiration
mean_aspiration_tpr = np.mean(tprs_aspiration, axis=0)
mean_aspiration_tpr[-1] = 1.0
aspiration_tpr_CI95 = [
        np.percentile(tprs_aspiration, 2.5, axis=0),
        np.percentile(tprs_aspiration, 97.5, axis=0)
        ]

mean_aspiration_auc = np.mean(auc_aspiration)
aspiration_auc_CI95 = [
        np.percentile(auc_aspiration, 2.5),
        np.percentile(auc_aspiration, 97.5)
        ]

aspiration_importances_df = pd.DataFrame(importances_aspiration)

mean_aspiration_fop = np.mean(aspiration_fops, axis=0)
aspiration_fop_CI95 = [
        np.percentile(aspiration_fops, 2.5, axis=0),
        np.percentile(aspiration_fops, 97.5, axis=0)
        ]
mean_aspiration_dws = np.mean(aspiration_dws, axis=0)
aspiration_dws_CI95 = [
        np.percentile(aspiration_dws, 2.5, axis=0),
        np.percentile(aspiration_dws, 97.5, axis=0)
        ]


# Sepsis
mean_sepsis_tpr = np.mean(tprs_sepsis, axis=0)
mean_sepsis_tpr[-1] = 1.0
sepsis_tpr_CI95 = [
        np.percentile(tprs_sepsis, 2.5, axis=0),
        np.percentile(tprs_sepsis, 97.5, axis=0)
        ]

mean_sepsis_auc = np.mean(auc_sepsis)
sepsis_auc_CI95 = [
        np.percentile(auc_sepsis, 2.5),
        np.percentile(auc_sepsis, 97.5)
        ]

sepsis_importances_df = pd.DataFrame(importances_sepsis)

mean_sepsis_fop = np.mean(sepsis_fops, axis=0)
sepsis_fop_CI95 = [
        np.percentile(sepsis_fops, 2.5, axis=0),
        np.percentile(sepsis_fops, 97.5, axis=0)
        ]
mean_sepsis_dws = np.mean(sepsis_dws, axis=0)
sepsis_dws_CI95 = [
        np.percentile(sepsis_dws, 2.5, axis=0),
        np.percentile(sepsis_dws, 97.5, axis=0)
        ]


# For AUC plot. Matplotlib
targets = ['Pneumonia', 'CHF', 'Aspiration', 'Sepsis']
heights = [mean_pna_auc, mean_chf_auc, mean_aspiration_auc, mean_sepsis_auc]
CI95 = [
    [heights[0] - pna_auc_CI95[0], heights[1] - chf_auc_CI95[0], heights[2] - aspiration_auc_CI95[0], heights[3] - sepsis_auc_CI95[0]],
    [pna_auc_CI95[1] - heights[0], chf_auc_CI95[1] - heights[1], aspiration_auc_CI95[1] - heights[2], sepsis_auc_CI95[1] - heights[3]]
       ]

temp_list = []
for item1, item2, item3, item4 in zip(auc_pna, auc_chf, auc_aspiration, auc_sepsis):
    temp = {'Pneumonia': item1, 'CHF': item2, 'Aspiration': item3, 'Sepsis': item4}
    temp_list.append(temp)
heights_df = pd.DataFrame(temp_list)

#### ROC curves and AUROCs for all models

In [ ]:
fig, ax = plt.subplots()

ax.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax.plot(mean_fpr, mean_pna_tpr, label=f"Pneumonia {heights[0]:.2f} ({pna_auc_CI95[0]:.2f}-{pna_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, pna_tpr_CI95[0], pna_tpr_CI95[1], alpha=0.2)

ax.plot(mean_fpr, mean_chf_tpr, label=f"CHF {heights[1]:.2f} ({chf_auc_CI95[0]:.2f}-{chf_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, chf_tpr_CI95[0], chf_tpr_CI95[1], alpha=0.2)

ax.plot(mean_fpr, mean_aspiration_tpr, label=f"Aspiration {heights[2]:.2f} ({aspiration_auc_CI95[0]:.2f}-{aspiration_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, aspiration_tpr_CI95[0], aspiration_tpr_CI95[1], alpha=0.2)

ax.plot(mean_fpr, mean_sepsis_tpr, label=f"Sepsis {heights[3]:.2f} ({sepsis_auc_CI95[0]:.2f}-{sepsis_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, sepsis_tpr_CI95[0], sepsis_tpr_CI95[1], alpha=0.2)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax.grid(linestyle=':')
ax.legend(loc='best', frameon=False)

fig.tight_layout()
# plt.savefig(figure_path / 'fig4_notes_roc.png')
plt.show()

In [ ]:
fig1, ax1 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

sns.pointplot(data=heights_df, errorbar=("pi", 95), ax=ax1,
              capsize=0.1, color="0", linestyle="none")
sns.swarmplot(data=heights_df, ax=ax1,
              palette=['tab:blue', 'tab:orange', 'tab:green', 'tab:red'],
              alpha=0.5, size=1.8)

ax1.set_ylabel('Performance (AUROC)')
ax1.set_xlabel('')
ax1.grid(linestyle=':', axis='y')
ax1.set_ylim(0.4, 1.0)
fig1.tight_layout()
# plt.savefig(figure_path / 'fig4_auc_notes.png')
plt.show()

#### Feature importances

In [ ]:
fig2, ax2 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

top15_pna = (
    pna_importances_df.groupby('feature')['importance'].mean().nlargest(15, keep="all").index
)
pna_importances_df_filtered = pna_importances_df[
    pna_importances_df['feature'].isin(top15_pna)
    ].sort_values(by='importance', ascending=False)
pna_importances_df_filtered['risk_factor'] = ["Pneumonia"]*len(pna_importances_df_filtered)

sns.pointplot(data=pna_importances_df_filtered,
              x="importance", y="feature", errorbar=("pi", 95), ax=ax2,
              capsize=0.1, color="0", linestyle="none")
sns.stripplot(data=pna_importances_df_filtered, size=3,
              x="importance", y="feature", ax=ax2, color="tab:blue", alpha=0.5)

ax2.set_xlabel('Importance')
ax2.set_ylabel('')
ax2.grid(linestyle=':', axis='x')
custom_legend = [Line2D([0], [0], marker='o', color='w', label='Pneumonia',
                        markerfacecolor='tab:blue', markersize=7, alpha=0.5)]
ax2.legend(handles=custom_legend, loc='lower right', frameon=False)
fig2.tight_layout()
# plt.savefig(figure_path / 'fig4_importance_pneumonia.png')
plt.show()

In [ ]:
fig3, ax3 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

top15_chf = (
    chf_importances_df.groupby('feature')['importance'].mean().nlargest(15, keep="all").index
)
chf_importances_df_filtered = chf_importances_df[
    chf_importances_df['feature'].isin(top15_chf)
    ].sort_values(by='importance', ascending=False)

sns.pointplot(data=chf_importances_df_filtered,
              x="importance", y="feature", errorbar=("pi", 95), ax=ax3,
              capsize=0.1, color="0", linestyle="none")
sns.stripplot(data=chf_importances_df_filtered, size=3,
              x="importance", y="feature", ax=ax3, color='tab:orange', alpha=0.5)

ax3.set_xlabel('Importance')
ax3.set_ylabel('')
ax3.grid(linestyle=':', axis='x')
custom_legend = [Line2D([0], [0], marker='o', color='w', label='CHF',
                        markerfacecolor='tab:orange', markersize=7, alpha=0.5)]
ax3.legend(handles=custom_legend, loc='lower right', frameon=False)
fig3.tight_layout()
# plt.savefig(figure_path / 'fig4_importance_chf.png')
plt.show()

In [ ]:
fig4, ax4 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

top15_aspiration = (
    aspiration_importances_df.groupby('feature')['importance'].mean().nlargest(15, keep="all").index
)
aspiration_importances_df_filtered = aspiration_importances_df[
    aspiration_importances_df['feature'].isin(top15_aspiration)
    ].sort_values(by='importance', ascending=False)

sns.pointplot(data=aspiration_importances_df_filtered,
              x="importance", y="feature", errorbar=("pi", 95), ax=ax4,
              capsize=0.1, color="0", linestyle="none")
sns.stripplot(data=aspiration_importances_df_filtered, size=3,
              x="importance", y="feature", ax=ax4, color='tab:green', alpha=0.5)

ax4.set_xlabel('Importance')
ax4.set_ylabel('')
ax4.grid(linestyle=':', axis='x')
custom_legend = [Line2D([0], [0], marker='o', color='w', label='Aspiration',
                        markerfacecolor='tab:green', markersize=7, alpha=0.5)]
ax4.legend(handles=custom_legend, loc='lower right', frameon=False)
fig4.tight_layout()
# plt.savefig(figure_path / 'fig4_importance_aspiration.png')
plt.show()

In [ ]:
fig5, ax5 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

top15_sepsis = (
    sepsis_importances_df.groupby('feature')['importance'].mean().nlargest(15, keep="all").index
)
sepsis_importances_df_filtered = sepsis_importances_df[
    sepsis_importances_df['feature'].isin(top15_sepsis)
    ].sort_values(by='importance', ascending=False)

sns.pointplot(data=sepsis_importances_df_filtered,
              x="importance", y="feature", errorbar=("pi", 95), ax=ax5,
              capsize=0.1, color="0", linestyle="none")
sns.stripplot(data=sepsis_importances_df_filtered, size=3,
              x="importance", y="feature", ax=ax5, color='tab:red', alpha=0.5)

ax5.set_xlabel('Importance')
ax5.set_ylabel('')
ax5.grid(linestyle=':', axis='x')
custom_legend = [Line2D([0], [0], marker='o', color='w', label='Sepsis',
                        markerfacecolor='tab:red', markersize=7, alpha=0.5)]
ax5.legend(handles=custom_legend, loc='lower right', frameon=False)
fig5.tight_layout()
# plt.savefig(figure_path / 'fig4_importance_sepsis.png')
plt.show()

#### Calibration curves

In [ ]:
fig6, ax6 = plt.subplots(4, 1, figsize=plots.stdfigsize(0, n_rows=4, n_cols=1, layout="single"))

# Pneumonia
ax6[0].plot(mean_mpv, mean_pna_fop, marker="o",
            label=f"Pneumonia\nDW (95% CI) = {mean_pna_dws:.2f} ({pna_dws_CI95[0]:.2f}-{pna_dws_CI95[1]:.2f})"
             )
ax6[0].fill_between(mean_mpv, pna_fop_CI95[0], pna_fop_CI95[1], alpha=0.2)

# Congestive Heart Failure
ax6[1].plot(mean_mpv, mean_chf_fop, marker="o",
            label=f"CHF\nDW (95% CI) = {mean_chf_dws:.2f} ({chf_dws_CI95[0]:.2f}-{chf_dws_CI95[1]:.2f})"
             )
ax6[1].fill_between(mean_mpv, chf_fop_CI95[0], chf_fop_CI95[1], alpha=0.2)

# Aspiration
ax6[2].plot(mean_mpv, mean_aspiration_fop, marker="o",
            label=f"Aspiration\nDW (95% CI) = {mean_aspiration_dws:.2f} ({aspiration_dws_CI95[0]:.2f}-{aspiration_dws_CI95[1]:.2f})"
             )
ax6[2].fill_between(mean_mpv, aspiration_fop_CI95[0], aspiration_fop_CI95[1], alpha=0.2)

# Sepsis
ax6[3].plot(mean_mpv, mean_sepsis_fop, marker="o",
            label=f"Sepsis\nDW (95% CI) = {mean_sepsis_dws:.2f} ({sepsis_dws_CI95[0]:.2f}-{sepsis_dws_CI95[1]:.2f})"
             )
ax6[3].fill_between(mean_mpv, sepsis_fop_CI95[0], sepsis_fop_CI95[1], alpha=0.2)


# Plot properties
ax6[0].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")
ax6[0].set_ylabel("Fraction of positive labels")
ax6[0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax6[0].grid(linestyle=':')
ax6[0].legend(loc='best', frameon=False)

ax6[1].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8)
ax6[1].set_ylabel("Fraction of positive labels")
ax6[1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax6[1].grid(linestyle=':')
ax6[1].legend(loc='best', frameon=False)

ax6[2].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8)
ax6[2].set_ylabel("Fraction of positive labels")
ax6[2].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax6[2].grid(linestyle=':')
ax6[2].legend(loc='best', frameon=False)

ax6[3].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8)
ax6[3].set_ylabel("Fraction of positive labels")
ax6[3].set_xlabel("Mean predicted probability")
ax6[3].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax6[3].grid(linestyle=':')
ax6[3].legend(loc='best', frameon=False)

fig6.tight_layout()
# plt.savefig(figure_path / "fig4_calibration_notes.png")
plt.show()

### Main panel/figure

In [ ]:
fig7, ax7 = plt.subplots(3, 2, figsize=plots.stdfigsize(128, n_rows=3, n_cols=2, layout="double"))

# Pneumonia ROC curve
ax7[0,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="No-skill line")

ax7[0,0].plot(mean_fpr, mean_pna_tpr, color='tab:blue', label=f"Pneumonia\nMean AUROC: {heights[0]:.2f}")
ax7[0,0].fill_between(mean_fpr, pna_tpr_CI95[0], pna_tpr_CI95[1], color='tab:blue', alpha=0.2)

# ax7[0,0].plot(mean_fpr, mean_chf_tpr, color='tab:orange', label=f"CHF {heights[1]:.2f}")
# ax7[0,0].fill_between(mean_fpr, chf_tpr_CI95[0], chf_tpr_CI95[1], color='tab:orange', alpha=0.2)

# ax7[0,0].plot(mean_fpr, mean_aspiration_tpr, color='tab:green', label=f"Aspiration {heights[2]:.2f}")
# ax7[0,0].fill_between(mean_fpr, aspiration_tpr_CI95[0], aspiration_tpr_CI95[1], color='tab:green' alpha=0.2)

# ax7[0,0].plot(mean_fpr, mean_sepsis_tpr, color='tab:red', label=f"Sepsis {heights[3]:.2f}")
# ax7[0,0].fill_between(mean_fpr, sepsis_tpr_CI95[0], sepsis_tpr_CI95[1], color='tab:red', alpha=0.2)

ax7[0,0].set_ylabel("True Positive Rate")
ax7[0,0].set_xlabel("False Positive Rate")
ax7[0,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax7[0,0].grid(linestyle=':')
ax7[0,0].legend(loc='best', frameon=False)
ax7[0,0].text(-0.15, 1.05, "a", transform=ax7[0,0].transAxes,
              fontweight='bold', va='top')


# AUCs of different models
sns.pointplot(data=heights_df, errorbar=("pi", 95), ax=ax7[0,1],
              capsize=0.1, color="0", linestyle="none")
sns.swarmplot(data=heights_df, ax=ax7[0,1],
              palette=['tab:blue', 'tab:orange', 'tab:green', 'tab:red'],
              alpha=0.5, size=1.5)

ax7[0,1].set_xlabel('')
ax7[0,1].set_ylabel('Performance (AUROC)')
ax7[0,1].grid(linestyle=':', axis='y')
ax7[0,1].set_ylim(0.4, 1.0)
ax7[0,1].text(-0.15, 1.05, "b", transform=ax7[0,1].transAxes,
              fontweight='bold', va='top')


# Pneumonia and Sepsis feature importances
sns.pointplot(data=pna_importances_df_filtered,
              x="importance", y="feature", errorbar=("pi", 95), ax=ax7[1,0],
              capsize=0.1, color="0", linestyle="none")
sns.stripplot(data=pna_importances_df_filtered,
              x="importance", y="feature", ax=ax7[1,0], color="tab:blue", alpha=0.5)
ax7[1,0].set_xlabel('Importance')
ax7[1,0].set_ylabel('')
ax7[1,0].grid(linestyle=':', axis='x')
custom_legend = [Line2D([0], [0], marker='o', color='w', label='Pneumonia',
                        markerfacecolor='tab:blue', markersize=8, alpha=0.5)]
ax7[1,0].legend(handles=custom_legend, loc='lower right', frameon=False)
ax7[1,0].text(-0.15, 1.05, "c", transform=ax7[1,0].transAxes,
              fontweight='bold', va='top')

sns.pointplot(data=sepsis_importances_df_filtered,
              x="importance", y="feature", errorbar=("pi", 95), ax=ax7[1,1],
              capsize=0.1, color="0", linestyle="none")
sns.stripplot(data=sepsis_importances_df_filtered,
              x="importance", y="feature", ax=ax7[1,1], color='tab:red', alpha=0.5)
ax7[1,1].set_xlabel('Importance')
ax7[1,1].set_ylabel('')
ax7[1,1].grid(linestyle=':', axis='x')
custom_legend = [Line2D([0], [0], marker='o', color='w', label='Sepsis',
                        markerfacecolor='tab:red', markersize=8, alpha=0.5)]
ax7[1,1].legend(handles=custom_legend, loc='best', frameon=False)


# Pneumonia and Sepsis calibration
ax7[2,0].plot(
    mean_mpv, mean_pna_fop, marker="o",
    label=f"Pneumonia\nDW = {mean_pna_dws:.2f}",
    color='tab:blue'
    )
ax7[2,0].fill_between(mean_mpv, pna_fop_CI95[0], pna_fop_CI95[1], color='tab:blue', alpha=0.2)
ax7[2,0].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")
ax7[2,0].set_xlabel("Mean predicted probability")
ax7[2,0].set_ylabel("Fraction of positive labels")
ax7[2,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax7[2,0].grid(linestyle=':')
ax7[2,0].legend(loc='best', frameon=False)
ax7[2,0].text(-0.15, 1.05, "d", transform=ax7[2,0].transAxes,
              fontweight='bold', va='top')

ax7[2,1].plot(
    mean_mpv, mean_sepsis_fop, marker="o",
    label=f"Sepsis\nDW = {mean_sepsis_dws:.2f}",
    color='tab:red'
    )
ax7[2,1].fill_between(mean_mpv, sepsis_fop_CI95[0], sepsis_fop_CI95[1], color='tab:red', alpha=0.2)
ax7[2,1].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")
ax7[2,1].set_xlabel("Mean predicted probability")
ax7[2,1].set_ylabel("")
ax7[2,1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax7[2,1].grid(linestyle=':')
ax7[2,1].legend(loc='best', frameon=False)

fig7.tight_layout()
# plt.savefig(figure_path / "fig4.png")
# plt.savefig(figure_path / "fig4.pdf")
plt.show()

## Generalization to MIMIC-III

### Training models

In [ ]:
# Train XGBoost on Hospital A (2013) notes that mention pneumonia
X_pna = pneumonia_notes[cols['a'][0]].to_numpy()
Y_pna = pneumonia_notes[cols['a'][1]].to_numpy()

# Vectorize
vect_pna = CountVectorizer(
    tokenizer=tokenizer_better,
    ngram_range=(1, 2),
    max_features=200
    )

vect_pna.fit(X_pna)
X_pna_vect = vect_pna.transform(X_pna).toarray()

model_pna = XGBClassifier(
    **pna_hyperparams,
    tree_method='hist',
    random_state=0
    )

model_pna.fit(X_pna_vect, Y_pna)

In [ ]:
# # Write vect_pna and model_pna to disk
# import pickle

# with open("src/pneumonia_model.pkl", "wb") as model_file:
#     pickle.dump(model_pna, model_file)
    
# with open("src/pneumonia_model_vectorizer.pkl", "wb") as vect_file:
#     pickle.dump(vect_pna, vect_file)

### Testing on MIMIC-III

In [ ]:
mean_fpr = np.arange(0, 1.01, 0.01)
mean_mpv = np.arange(0, 1.1, 0.1)

In [ ]:
aucs_pna = []
tprs_pna = []
fops_pna = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_segmented = pneumonia_notes_mimic.sample(
        n=len(pneumonia_notes_mimic),
        replace=True,
        axis=0
        )
    
    encounters = boot_segmented['encounter_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        model_pna,
        boot_segmented,
        {'a': ['seg_pneumonia', 'curt_pneumonia_(1=yes)']},
        'a',
        encounters,
        vect_pna,
        train_index,
        mean_fpr,
        mean_mpv
        ) for train_index, test_index in cv.split(encounters))
    
    aucs_pna.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    tprs_pna.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_pna.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

#### ROC and Calibration plots

In [ ]:
mean_tpr_pna = np.mean(tprs_pna, axis=0)
mean_tpr_pna[-1] = 1.0  
tpr_pna_CI95 = [
    np.percentile(tprs_pna, 25, axis=0),
    np.percentile(tprs_pna, 75, axis=0)
    ]

mean_auc_pna = np.mean(aucs_pna)
auc_pna_CI95 = [
    np.percentile(aucs_pna, 2.5),
    np.percentile(aucs_pna, 97.5)
    ]

mean_fop_pna = np.mean(fops_pna, axis=0)
fop_pna_CI95 = [
    np.percentile(fops_pna, 2.5, axis=0),
    np.percentile(fops_pna, 97.5, axis=0)
    ]

In [ ]:
fig8, ax8 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

ax8.plot(
    mean_fpr,
    mean_tpr_pna,
    label=f"Mean AUROC: {mean_auc_pna:.3f}"
    )
ax8.fill_between(
    mean_fpr,
    tpr_pna_CI95[0],
    tpr_pna_CI95[1],
    alpha=0.2
    )

ax8.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="No-skill line")

ax8.set_ylabel("True Positive Rate")
ax8.set_xlabel("False Positive Rate")
ax8.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax8.grid(linestyle=':')
ax8.legend(loc='best', frameon=False)
ax8.text(-0.15, 1.05, "a", transform=ax8.transAxes,
         fontweight='bold', va='top')

fig8.tight_layout()
# plt.savefig(figure_path / 'fig5_ROC.png')
plt.show()

In [ ]:
fig9, ax9 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

ax9.plot(mean_mpv, mean_fop_pna, marker="o")
ax9.fill_between(
    mean_mpv, fop_pna_CI95[0], fop_pna_CI95[1], alpha=0.2)
ax9.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")

ax9.tick_params(axis='x')
ax9.tick_params(axis='y')
ax9.set_xlabel("Mean predicted probability")
ax9.set_ylabel("Fraction of positive labels")
ax9.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax9.grid(linestyle=':')
ax9.legend(loc='best', frameon=False)
ax9.text(-0.15, 1.05, "c", transform=ax9.transAxes,
         fontweight='bold', va='top')

fig9.tight_layout()
# plt.savefig(figure_path / 'fig5_calibration.png')
plt.show()

#### SHAP values

In [ ]:
X = pneumonia_notes_mimic['seg_pneumonia'].to_numpy()
Y = pneumonia_notes_mimic['curt_pneumonia_(1=yes)'].to_numpy()

X_vect = vect_pna.transform(X).toarray()

In [ ]:
explainer = shap.explainers.Tree(
    model_pna,
    X_vect,
    feature_perturbation='interventional',
    feature_names=vect_pna.get_feature_names_out(),
    model_output='probability'
    )

shap_values = explainer(X_vect)

In [ ]:
fig10, ax10 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))

shap.plots.beeswarm(shap_values, max_display=15, show=False, s=3,
                    ax=ax10, plot_size=None, group_remaining_features=False)

ax10.set_xlabel("SHAP values", fontsize=6)
ax10.tick_params(axis='both', which='both', labelsize=6)
ax10.set_xlim(-0.6, 0.6)
ax10.text(-0.45, 1.05, "b", transform=ax10.transAxes,
          fontweight='bold', va='top')

cb1 = plt.gcf().get_axes()[1]
cb1.set_yticklabels(['0', str(X_vect.max())], fontsize=6)
cb1.set_ylabel("Word count in note", fontsize=6)

plt.tight_layout()
# plt.savefig(figure_path / "fig5_SHAP.png")
plt.show()

#### Confusion matrix

In [ ]:
pneumonia_notes_mimic['predicted'] = model_pna.predict(X_vect)

cf = confusion_matrix(Y, pneumonia_notes_mimic['predicted'])
cf_mod = cf.transpose()[::-1, ::-1]

strings = np.asarray([['True positives\n', 'False positives\n'],
                      ['False negatives\n', 'True negatives\n']])

labels = (np.asarray(["{0} {1:.0f}".format(string, value)
                      for string, value in zip(strings.flatten(),
                                               cf_mod.flatten())])
         ).reshape(2, 2)

In [ ]:
fig11, ax11 = plt.subplots(figsize=plots.stdfigsize(0, layout="single"))
sns.heatmap(cf_mod, fmt='', annot=labels, cmap='Blues', cbar=False, ax=ax11)
ax11.set_ylabel("Pneumonia Model adjudicated")
ax11.set_xlabel("Ground truth")
ax11.tick_params(axis='both', bottom=False, left=False,
                 labelbottom=False, labelleft=False)
ax11.text(-0.1, 1.05, "d", transform=ax11.transAxes,
          fontweight='bold', va='top')

fig11.tight_layout()
# plt.savefig(figure_path / 'fig5_cf.png')
plt.show()

In [ ]:
fig12, ax12 = plt.subplots(2, 2, figsize=plots.stdfigsize(129, n_rows=2, n_cols=2, layout="double"))

ax12[0,0].plot(
    mean_fpr,
    mean_tpr_pna,
    label=f"Mean AUROC: {mean_auc_pna:.3f}"
    )
ax12[0,0].fill_between(
    mean_fpr,
    tpr_pna_CI95[0],
    tpr_pna_CI95[1],
    alpha=0.2
    )
ax12[0,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="No-skill line")

ax12[0,0].set_ylabel("True Positive Rate")
ax12[0,0].set_xlabel("False Positive Rate")
ax12[0,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax12[0,0].grid(linestyle=':')
ax12[0,0].legend(loc='lower right', frameon=False)
ax12[0,0].text(-0.15, 1.05, "a", transform=ax12[0,0].transAxes, fontweight='bold', va='top')


shap.plots.beeswarm(
    shap_values, max_display=15, show=False, s=3, ax=ax12[0,1],
    plot_size=None, group_remaining_features=False)

ax12[0,1].set_xlabel("SHAP values", fontsize=6)
ax12[0,1].tick_params(axis='both', which='both', labelsize=6)
ax12[0,1].set_xlim(-0.6, 0.6)
ax12[0,1].text(-0.35, 1.05, "b", transform=ax12[0,1].transAxes, fontweight='bold', va='top')
cb1 = plt.gcf().get_axes()[4]
cb1.set_yticklabels(['0', str(X_vect.max())], fontsize=6)
cb1.set_ylabel("Word count in note", fontsize=6)


ax12[1,0].plot(mean_mpv, mean_fop_pna, marker="o")
ax12[1,0].fill_between(
    mean_mpv, fop_pna_CI95[0], fop_pna_CI95[1], alpha=0.2)
ax12[1,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")

ax12[1,0].tick_params(axis='x')
ax12[1,0].tick_params(axis='y')
ax12[1,0].set_xlabel("Mean predicted probability")
ax12[1,0].set_ylabel("Fraction of positive labels")
ax12[1,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax12[1,0].grid(linestyle=':')
ax12[1,0].legend(loc='lower right', frameon=False)
ax12[1,0].text(-0.15, 1.05, "c", transform=ax12[1,0].transAxes, fontweight='bold', va='top')


sns.heatmap(cf_mod, fmt='', annot=labels, cmap='Blues', cbar=False, ax=ax12[1,1])
ax12[1,1].set_ylabel("Pneumonia Model adjudicated")
ax12[1,1].set_xlabel("Ground truth")
ax12[1,1].tick_params(axis='both', bottom=False, left=False, labelbottom=False, labelleft=False)
ax12[1,1].text(-0.15, 1.05, "d", transform=ax12[1,1].transAxes, fontweight='bold', va='top')


fig12.tight_layout()
# plt.savefig(figure_path / 'fig5.png')
# plt.savefig(figure_path / 'fig5.pdf')
plt.show()

### SI Figure on different choices for cutoff

In [ ]:
pneumonia_notes_mimic['pneumonia_probability'] = model_pna.predict_proba(X_vect)[:, 1]

In [ ]:
results = []
thresholds = np.linspace(0, 1, 1001)

for threshold in thresholds:
    predictions = pneumonia_notes_mimic['pneumonia_probability'] >= threshold
    
    cf = confusion_matrix(Y, predictions)
    TN = cf[0,0]
    FP = cf[0,1]
    FN = cf[1,0]
    TP = cf[1,1]
    
    results.append({
        'threshold': threshold,
        'false_negative_rate': round(FN / (FN + TP), 3),
        'false_positive_rate': round(FP / (FP + TN), 3),
        'precision': round(precision_score(Y, predictions), 3),
        'negative_predictive_value': round(TN / (TN + FN), 3),
        'accuracy': round(accuracy_score(Y, predictions), 3),
        'f1': round(f1_score(Y, predictions), 3),
        'youden_j': round((TP / (TP + FN)) + (TN / (TN + FP)) - 1, 3)
        })

In [ ]:
results_df = pd.DataFrame(results)
max_youden = results_df['youden_j'] == results_df['youden_j'].max()
max_f1 = results_df['f1'] == results_df['f1'].max()
max_accuracy = results_df['accuracy'] == results_df['accuracy'].max()

In [ ]:
results_df.loc[max_accuracy].sort_values('threshold', ascending=False)

In [ ]:
plt.plot(results_df['threshold'], results_df['accuracy'])

In [ ]:
results_df.loc[max_f1].sort_values('threshold', ascending=False)

In [ ]:
plt.plot(results_df['threshold'], results_df['f1'])

In [ ]:
results_df.loc[max_youden].sort_values('threshold', ascending=False)

In [ ]:
plt.plot(results_df['threshold'], results_df['youden_j'])

In [ ]:
results_df.loc[results_df['threshold'] == 0.5]